### Importing Relevant Libraries

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.core.display import HTML

### Initiating webdriver & Opening Website

In [7]:
website = "https://www.zameen.com"
driver = webdriver.Chrome("C:\Chrome Web Driver\chromedriver_win32\chromedriver.exe")

driver.get(website)

driver.maximize_window()

C:\Users\USER\AppData\Local\Temp\ipykernel_18028\1631413679.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\Chrome Web Driver\chromedriver_win32\chromedriver.exe")


### Navigating to Desired Page through Driver commands

In [8]:
# clicking city dropdown
driver.find_element(By.XPATH, '//*[@id="body-wrapper"]/header/div[6]/div/div[2]/div[2]/div[1]/div[1]/div/div').click()
time.sleep(1)

# Selecting(clicking) Karachi
driver.find_element(By.XPATH, '//*[@id="body-wrapper"]/header/div[6]/div/div[2]/div[2]/div[1]/div[1]/div/div[2]/div/div[1]/div/span[2]/button').click()

# clicking and searching 'Gulshan'
search = driver.find_element(By.CLASS_NAME, 'a41c4dcc')
search.click()
time.sleep(1)
search.send_keys('Gulshan')
time.sleep(1)
search.send_keys(Keys.ENTER)
time.sleep(2)

# clicking Find
driver.find_element(By.XPATH, '//*[@id="body-wrapper"]/header/div[6]/div/div[2]/div[2]/div[1]/a').click()

### Scrapping Data of all pages through Pagination 

    Scrapping data from one page in one iteration, then clicking next button and repeating the process until we reach the last page

In [9]:
dataFrame = {'Title': [] ,'Price': [], 'Location': [], 'Bed': [], 'Bath': [], 'Area': [], 'Badges': [], 'Listing Time': [], 'Thumbnail': []}

next_button = True

while next_button:
    
    # Scrapping Price
    prices = driver.find_elements(By.CLASS_NAME, 'f343d9ce')
    for i in prices:
        dataFrame['Price'].append(i.text)


    # Scrapping Location
    locations = driver.find_elements(By.CLASS_NAME, '_162e6469')
    for i in locations:
        dataFrame['Location'].append(i.text)


    # Scrapping Listing-Time
    listing = driver.find_elements(By.CLASS_NAME, '_08b01580')
    for i in listing:
        dataFrame['Listing Time'].append(i.text.replace('\n', " "))


    # Scrapping Images
    properties = driver.find_elements(By.XPATH, '//li[@role = "article"]')
    for i in properties:
        temp = i.find_element(By.CLASS_NAME, '_150e2e12').find_element(By.TAG_NAME, 'source').get_attribute("srcset")
        if temp == '':
            temp = i.find_element(By.CLASS_NAME, '_150e2e12').find_element(By.TAG_NAME, 'source').get_attribute("data-srcset")
        dataFrame['Thumbnail'].append(temp)


    # Scrapping Badges
    div_badges = driver.find_elements(By.CLASS_NAME, 'b75fc1d9')
    n = 1
    for i in div_badges:
        badges = i.find_elements(By.CLASS_NAME, '_38842953')
        temp = []
        for j in badges:
            temp.append(j.find_element(By.TAG_NAME, 'svg').get_attribute('aria-label'))
        dataFrame['Badges'].append(temp)
    for i in dataFrame['Badges']:
        try:
            if i[0] == None:
                i[0] = 'Hot badge'
        except:
            i.append('No badges')


    # Scrapping Title
    titles = driver.find_elements(By.CLASS_NAME, 'c0df3811')
    for i in titles:
        dataFrame['Title'].append(i.text)


    # Scrapping Bed,Bath,Area
    divs_BB = driver.find_elements(By.CLASS_NAME, '_27f6c93d')
    for i in divs_BB:
        temp = i.find_elements(By.CLASS_NAME, '_984949e5')
        for j in temp:
            if j.get_attribute('aria-label') == "Beds":
                dataFrame['Bed'].append(j.text)
            elif j.get_attribute('aria-label') == "Baths":
                dataFrame['Bath'].append(j.text)
            elif j.get_attribute('aria-label') == "Area":
                dataFrame['Area'].append(j.text)
        if len(dataFrame['Bed']) == len(dataFrame['Area'])-1:
            dataFrame['Bed'].append('NA')
        if len(dataFrame['Bath']) == len(dataFrame['Area'])-1:
            dataFrame['Bath'].append('NA')
        
        
    # checking if next button exists on this page
    next_button = (driver.find_elements(By.CLASS_NAME, 'b7880daf')[-1].get_attribute('title') == "Next")
    
    # clicking Next button
    element = driver.find_elements(By.CLASS_NAME, 'b7880daf')[-1]
    driver.execute_script("arguments[0].click();", element)
    time.sleep(5)

KeyboardInterrupt: 

### Making DataFrame of our Scrapped Data

In [11]:
df = pd.DataFrame(data = dataFrame)
df

,Title,Price,Location,Bed,Bath,Area,Badges,Listing Time,Thumbnail
0,250 Sqyds Villa Is Available On Easy Installments,2.9 Crore,"Gulshan-e-Iqbal, Gulshan-e-Iqbal Town",4.0,4.0,250 Sq. Yd.,"['Hot badge', 'Trusted badge']",Added: 18 hours ago (Updated: 18 hours ago),https://media.zameen.com/thumbnails/203549197-...
1,3 Bed D D Al Hamra Garden Bock 7 Gulshan-e-iqb...,1.2 Crore,"Gulshan-e-Iqbal - Block 7, Gulshan-e-Iqbal",3.0,3.0,150 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 1 week ago,https://media.zameen.com/thumbnails/202569126-...
2,A Prime Location 1170 Square Feet Flat Has Lan...,1.4 Crore,"Gulshan-e-Iqbal - Block 10-A, Gulshan-e-Iqbal",3.0,3.0,130 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 1 week ago,https://media.zameen.com/thumbnails/202307331-...
3,300 yards Double Story Corner Bungalow for Sale,6 Crore,"Gulshan-e-Jamal, Gulshan-e-Iqbal Town",8.0,6.0,325 Sq. Yd.,['Hot badge'],Added: 9 hours ago,https://media.zameen.com/thumbnails/195808768-...
4,Corner 1830 Square Feet Flat For sale Is Avail...,1.4 Crore,"Gulshan-e-Iqbal - Block 11, Gulshan-e-Iqbal",NaN,NaN,203 Sq. Yd.,"['Hot badge', 'Verified badge']",Added: 18 hours ago,https://media.zameen.com/thumbnails/203043826-...
...,...,...,...,...,...,...,...,...,...
1949,950 Square Feet Flat In Gulshan-E-Iqbal - Bloc...,80 Lakh,"Gulshan-e-Iqbal - Block 4, Gulshan-e-Iqbal",2.0,2.0,106 Sq. Yd.,['No badges'],Added: 9 months ago (Updated: 9 months ago),https://images.zameen.com/smap/400/300/24.9346...
1950,Get In Touch Now To Buy A 1000 Square Feet Fla...,1.1 Crore,"Gulshan-e-Iqbal - Block 5, Gulshan-e-Iqbal",2.0,2.0,111 Sq. Yd.,['No badges'],Added: 9 months ago (Updated: 9 months ago),https://images.zameen.com/smap/400/300/24.9194...
1951,This Is Your Chance To Buy Flat In Gulshan-E-I...,85 Lakh,"Gulshan-e-Iqbal - Block 2, Gulshan-e-Iqbal",2.0,2.0,83 Sq. Yd.,['No badges'],Added: 9 months ago (Updated: 9 months ago),https://images.zameen.com/smap/400/300/24.9240...
1952,House For Sale Gulshan-e-Iqbal - Block 5,4.5 Crore,"Gulshan-e-Iqbal - Block 5, Gulshan-e-Iqbal",5.0,5.0,240 Sq. Yd.,['No badges'],Added: 9 months ago (Updated: 9 months ago),https://images.zameen.com/smap/400/300/24.9194...


In [12]:
df.to_csv('Scrapped data-Zameen.csv', index = False)

### Converting Thumbnails(img links) to Images

In [13]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [14]:
df.to_html(escape=False, formatters=dict(Thumbnail=path_to_image_html))

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Title</th>\n      <th>Price</th>\n      <th>Location</th>\n      <th>Bed</th>\n      <th>Bath</th>\n      <th>Area</th>\n      <th>Badges</th>\n      <th>Listing Time</th>\n      <th>Thumbnail</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>250 Sqyds Villa Is Available On Easy Installments</td>\n      <td>2.9 Crore</td>\n      <td>Gulshan-e-Iqbal, Gulshan-e-Iqbal Town</td>\n      <td>4.0</td>\n      <td>4.0</td>\n      <td>250 Sq. Yd.</td>\n      <td>[\'Hot badge\', \'Trusted badge\']</td>\n      <td>Added: 18 hours ago (Updated: 18 hours ago)</td>\n      <td><img src="https://media.zameen.com/thumbnails/203549197-400x300.webp" width="60" ></td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>3 Bed D D Al Hamra Garden Bock 7 Gulshan-e-iqbal Near Sunday Bazar</td>\n      <td>1.2 Crore</td>\n      <td>Gulshan-e-Iqbal - Block 7, Gulshan-e-Iqbal<

In [21]:
df = HTML(df.to_html(escape=False,formatters=dict(Thumbnail=path_to_image_html)))
df

,Title,Price,Location,Bed,Bath,Area,Badges,Listing Time,Thumbnail
0,250 Sqyds Villa Is Available On Easy Installments,2.9 Crore,"Gulshan-e-Iqbal, Gulshan-e-Iqbal Town",4.0,4.0,250 Sq. Yd.,"['Hot badge', 'Trusted badge']",Added: 18 hours ago (Updated: 18 hours ago),
1,3 Bed D D Al Hamra Garden Bock 7 Gulshan-e-iqbal Near Sunday Bazar,1.2 Crore,"Gulshan-e-Iqbal - Block 7, Gulshan-e-Iqbal",3.0,3.0,150 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 1 week ago,
2,A Prime Location 1170 Square Feet Flat Has Landed On Market In Gulshan-e-Iqbal - Block 10-A Of Karachi,1.4 Crore,"Gulshan-e-Iqbal - Block 10-A, Gulshan-e-Iqbal",3.0,3.0,130 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 1 week ago,
3,300 yards Double Story Corner Bungalow for Sale,6 Crore,"Gulshan-e-Jamal, Gulshan-e-Iqbal Town",8.0,6.0,325 Sq. Yd.,['Hot badge'],Added: 9 hours ago,
4,Corner 1830 Square Feet Flat For sale Is Available In Gulshan-e-Iqbal - Block 11,1.4 Crore,"Gulshan-e-Iqbal - Block 11, Gulshan-e-Iqbal",NaN,NaN,203 Sq. Yd.,"['Hot badge', 'Verified badge']",Added: 18 hours ago,
5,Brand New Portion Available For Sale,4.25 Crore,"Bahadurabad, Gulshan-e-Iqbal Town",4.0,4.0,244 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 21 hours ago,
6,Portion Available For Sale Near 4 Minar Chowrangi,4.2 Crore,"Sharfabad, Gulshan-e-Iqbal Town",5.0,6.0,267 Sq. Yd.,"['Hot badge', 'Trusted badge', 'Verified badge']",Added: 21 hours ago,
7,3 Bed D/D Fully Renovated Apartment,1.1 Crore,"Gulshan-e-Iqbal - Block 13/D-2, Gulshan-e-Iqbal",3.0,3.0,139 Sq. Yd.,"['Hot badge', 'Verified badge']",Added: 22 hours ago,
8,2 Bed Fully Renovated Apartment,65 Lakh,"Gulshan-e-Iqbal - Block 13/D-2, Gulshan-e-Iqbal",2.0,2.0,83 Sq. Yd.,"['Hot badge', 'Verified badge']",Added: 22 hours ago,
9,A Perfect Prime Location Flat Awaits You In Gulshan-e-Iqbal - Block 4A Karachi,1.95 Crore,"Gulshan-e-Iqbal - Block 4A, Gulshan-e-Iqbal",NaN,NaN,167 Sq. Yd.,"['Hot badge', 'Verified badge']",Added: 2 days ago,
